In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../../data/'
group_dir_ = os.path.join(dir_, 'groups/high')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

220

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '20']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
pop_dict = {}
for i in df['tid'].unique():
    pop_dict[i] = len(df[df['tid']==i])/num_user

In [5]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,3,2.544071,1
1,0,7,2.545822,1
2,0,18,2.152490,1
3,0,20,1.882396,1
4,0,31,2.410806,1


In [6]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)
    
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
166,166,3.840909
7209,7209,3.295455
350,350,3.113636
3761,3761,3.045455
80,80,3.022727
1017,1017,2.840909
1753,1753,2.750000
428,428,2.727273
200,200,2.681818
3159,3159,2.659091


In [7]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [9]:
for lambda_ in tqdm(lambdas):
    top_n_size = 50
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    threshold = 0.0
    avg_pops = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        pops = []
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j]
            for k in range(len(group)):
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    pops.append(pop_dict[tid])                    
        avg_pop = sum(pops)/len(pops)
        avg_pops.append(avg_pop)
        
    print(avg_pops)

[0.5025454545454436, 0.5025454545454445, 0.5025454545454486, 0.502545454545453, 0.5025454545454555, 0.5025454545454572, 0.5025454545454615, 0.5025454545454616, 0.5025454545454615, 0.5025454545454616]
[0.500219008264454, 0.500849090909083, 0.5012639296187633, 0.5014025974025961, 0.501755681818183, 0.5017202797202819, 0.5018090909090969, 0.5017500000000066, 0.5018863636363698, 0.502227272727279]
[0.4889834710743774, 0.4922618181818137, 0.4943167155425193, 0.49517748917749044, 0.4958465909090941, 0.4967832167832204, 0.49713636363637087, 0.4966363636363699, 0.4978636363636428, 0.49954545454545785]
[0.4709948347107457, 0.4738690909090885, 0.4777624633431086, 0.4802943722943755, 0.48275000000000423, 0.4824475524475566, 0.4840818181818244, 0.486431818181823, 0.48414772727273647, 0.49481818181818255]
[0.44509710743802083, 0.4474272727272714, 0.4484046920821114, 0.4512337662337682, 0.4529374999999977, 0.45627272727272894, 0.45805454545454866, 0.46106818181818027, 0.46535227272727975, 0.48145454